# Sensitivity Analysis: 1D, 2D Sweeps and Ablations
Use the `SensitivityRunner` to sweep device parameters and summarize Quantum Volume metrics.

In [ ]:
# Setup imports
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import display
root = Path.cwd()
src_path = root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
from spinq_qv.config.schemas import Config
from spinq_qv.experiments.run_qv import run_experiment
from spinq_qv.experiments.sensitivity import SensitivityRunner, create_default_param_ranges

In [ ]:
# Create base config and runner
baseline_yaml = root / "examples" / "configs" / "baseline.yaml"
config = Config.from_yaml(baseline_yaml)
config.simulation.widths = [2,3]
config.simulation.n_circuits = 5
config.simulation.n_shots = 200
runner = SensitivityRunner(config, output_dir=root / "results" / "sensitivity_demo")
param_ranges = create_default_param_ranges(config)
param_ranges

In [ ]:
# 1D sweep example: vary F2 across 4 values
values = param_ranges['F2']
df_f2 = runner.run_parameter_sweep(
    parameter_path=['device','F2'],
    values=values,
    qv_runner_func=lambda cfg, **kw: run_experiment(cfg, root / 'results' / 'sensitivity_demo' / 'F2', seed=cfg.simulation.random_seed, parallel=False, n_workers=1, enable_profiling=False, return_aggregated=True),
)
df_f2.head()

In [ ]:
# 2D grid example: F1 vs T2 (downsized for demo)
param1 = ['device','F1']; v1 = param_ranges['F1']
param2 = ['device','T2']; v2 = param_ranges['T2']
df_grid, grid = runner.run_2d_grid(
    param1, v1, param2, v2,
    qv_runner_func=lambda cfg, **kw: run_experiment(cfg, root / 'results' / 'sensitivity_demo' / 'grid', seed=cfg.simulation.random_seed, parallel=False, n_workers=1, enable_profiling=False, return_aggregated=True),
)
grid.shape

In [ ]:
# Ablation sweep example
df_abl = runner.run_ablation_sweep(
    qv_runner_func=lambda cfg, **kw: run_experiment(cfg, root / 'results' / 'sensitivity_demo' / 'ablation', seed=cfg.simulation.random_seed, parallel=False, n_workers=1, enable_profiling=False, return_aggregated=True),
)
df_abl.head()